In [2]:
import json

In [3]:
from elasticsearch import Elasticsearch

# Define Elasticsearch Cloud credentials
cloud_id = "finalproject:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ0NTI4OTc0ZTg5MmU0ZmY2YmU3NDNmODUzYTI1MDhjOSQyODdmZTVmMTc0MmM0ZWNmODgzN2M1YTNmNGE5MWQ1ZA=="
cloud_auth = ("elastic", "SvXQmBxTZWeUQC88AloilfrY")  # Credentials for accessing Elasticsearch Cloud

# Create Elasticsearch client with the specified parameters
es = Elasticsearch(cloud_id=cloud_id, basic_auth=cloud_auth)

# Check if the connection is successful
if es.ping():
    print("Connected to Elasticsearch Cloud")
else:
    print("Connection failed")



Connected to Elasticsearch Cloud


In [27]:
index_name = "tweeeets5"



# Define the mapping
mapping = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 2000
    },
    "mappings": {
        "properties": {
            "created_at": {"type": "date", "format": "EEE MMM dd HH:mm:ss Z yyyy"},
            "id": {"type": "long"},
            "id_str": {"type": "keyword"},
            "text": {"type": "text", "fielddata": True},
            "user": {
                "properties": {
                    "id": {"type": "long"},
                    "id_str": {"type": "keyword"},
                    "name": {"type": "text"},
                    "screen_name": {"type": "keyword"},
                  #  "dynamic": "false"
                }
            },
            "retweeted_status": {
                "properties": {
                    "created_at": {"type": "date", "format": "EEE MMM dd HH:mm:ss Z yyyy"},
                    "id": {"type": "long"},
                    "id_str": {"type": "keyword"},
                    "text": {"type": "text", "fielddata": True},
                    "user": {
                        "properties": {
                            "id": {"type": "long"},
                            "id_str": {"type": "keyword"},
                            "name": {"type": "text"},
                            "screen_name": {"type": "keyword"}
                        }
                    },
                    "retweet_count": {"type": "integer"},
                    "favorite_count": {"type": "integer"},
                    "favorited": {"type": "boolean"},
                    "retweeted": {"type": "boolean"},
                    "possibly_sensitive": {"type": "boolean"}
                }
            },
            "retweet_count": {"type": "integer"},
            "favorite_count": {"type": "integer"},
            "favorited": {"type": "boolean"},
            "retweeted": {"type": "boolean"},
            "possibly_sensitive": {"type": "boolean"}
        }
    }
}

# Create the index with the mapping
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping, ignore=400)  # ignore 400 already exists code


In [35]:
def preparedTweet(tweet):
    prepared_tweet = {
                "created_at": tweet.get("created_at"),
                "id": tweet.get("id"),
                "id_str": tweet.get("id_str"),
                "text": tweet.get("text"),
                "user": {
                    "id": user_info.get("id"),
                    "id_str": user_info.get("id_str"),
                    "name": user_info.get("name"),
                    "screen_name": user_info.get("screen_name")
                },
                "retweeted_status": {
                    "created_at": retweeted_status.get("created_at"),
                    "id": retweeted_status.get("id"),
                    "id_str": retweeted_status.get("id_str"),
                    "text": retweeted_status.get("text"),
                    "user": {
                        "id": retweeted_user_info.get("id"),
                        "id_str": retweeted_user_info.get("id_str"),
                        "name": retweeted_user_info.get("name"),
                        "screen_name": retweeted_user_info.get("screen_name")
                    },
                    "retweet_count": retweeted_status.get("retweet_count"),
                    "favorite_count": retweeted_status.get("favorite_count"),
                    "favorited": retweeted_status.get("favorited"),
                    "retweeted": retweeted_status.get("retweeted"),
                    "possibly_sensitive": retweeted_status.get("possibly_sensitive")
                } if retweeted_status else {},
                "retweet_count": tweet.get("retweet_count"),
                "favorite_count": tweet.get("favorite_count"),
                "favorited": tweet.get("favorited"),
                "retweeted": tweet.get("retweeted"),
                "possibly_sensitive": tweet.get("possibly_sensitive")
            }
    return prepared_tweet

In [33]:
index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.mapping.total_fields.limit": 1000,  # Limiting the number of fields
    },
    "mappings": {
        "properties": {
            "tweet_id": {"type": "keyword"},  # Using keyword for exact matches
        }
    }
}

# Create the index
index_name_tweetids = "tweet_ids"
if not es.indices.exists(index=index_name_tweetids):
    es.indices.create(index=index_name_tweetids, body=index_body)
    print(f"Index '{index_name_tweetids}' created.")
else:
    print(f"Index '{index_name_tweetids}' already exists.")

Index 'tweet_ids' created.


In [32]:
def index_tweet(tweet):
    res = es.index(index=index_name, id=tweet["id"], document=tweet)
    print(f"Indexed Tweet ID: {tweet['id']} Status: {res['result']}")
import time

start_time = time.time()
# Read the file and index each tweet
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", 'r') as file:
    for line in file:
        if not line.strip():
            continue
        tweet = json.loads(line.strip())
        tweet_id = tweet.get("id_str")
        if es.exists(index=index_name, id=tweet_id):
                print(f"Tweet ID {tweet_id} already exists.")
                continue
        user_info = tweet.get("user", {})
        retweeted_status = tweet.get("retweeted_status", {})
        retweeted_user_info = retweeted_status.get("user", {}) if retweeted_status else {}
        es.index(index=index_name_tweetids, id=tweet_id, document={"tweet_id": tweet_id, "user_id": user_id}})
        prepared_tweet = {
            "created_at": tweet.get("created_at"),
            "id": tweet.get("id"),
            "id_str": tweet.get("id_str"),
            "text": tweet.get("text"),
            "user": {
                "id": user_info.get("id"),
                "id_str": user_info.get("id_str"),
                "name": user_info.get("name"),
                "screen_name": user_info.get("screen_name")
            },
            "retweeted_status": {
                "created_at": retweeted_status.get("created_at"),
                "id": retweeted_status.get("id"),
                "id_str": retweeted_status.get("id_str"),
                "text": retweeted_status.get("text"),
                "user": {
                    "id": retweeted_user_info.get("id"),
                    "id_str": retweeted_user_info.get("id_str"),
                    "name": retweeted_user_info.get("name"),
                    "screen_name": retweeted_user_info.get("screen_name")
                },
                "retweet_count": retweeted_status.get("retweet_count"),
                "favorite_count": retweeted_status.get("favorite_count"),
                "favorited": retweeted_status.get("favorited"),
                "retweeted": retweeted_status.get("retweeted"),
                "possibly_sensitive": retweeted_status.get("possibly_sensitive")
            } if retweeted_status else {},
            "retweet_count": tweet.get("retweet_count"),
            "favorite_count": tweet.get("favorite_count"),
            "favorited": tweet.get("favorited"),
            "retweeted": tweet.get("retweeted"),
            "possibly_sensitive": tweet.get("possibly_sensitive")
        }
        index_tweet(prepared_tweet)

end_time = time.time()  # Capture end time
print(f"Total runtime of the program is {end_time - start_time} seconds")
print("All tweets have been indexed.")

Indexed Tweet ID: 1249403767180668930 Status: updated
Indexed Tweet ID: 1249403768023678982 Status: updated
Indexed Tweet ID: 1249403769193779202 Status: updated
Indexed Tweet ID: 1249403769567227906 Status: updated
Indexed Tweet ID: 1249403770435493888 Status: updated
Indexed Tweet ID: 1249403770360016896 Status: created
Indexed Tweet ID: 1249403770833981440 Status: created
Indexed Tweet ID: 1249403771077005315 Status: created
Indexed Tweet ID: 1249403771261722624 Status: created
Indexed Tweet ID: 1249403771676815361 Status: created
Indexed Tweet ID: 1249403771781816323 Status: created
Indexed Tweet ID: 1249403772679438340 Status: created
Indexed Tweet ID: 1249403773178560515 Status: created
Indexed Tweet ID: 1249403773476356096 Status: created
Indexed Tweet ID: 1249403773522337792 Status: created
Indexed Tweet ID: 1249403773740552193 Status: created
Indexed Tweet ID: 1249403773799170049 Status: created
Indexed Tweet ID: 1249403774151659521 Status: created
Indexed Tweet ID: 1249403774

Indexed Tweet ID: 1249403815494709248 Status: created
Indexed Tweet ID: 1249403815587127296 Status: created
Indexed Tweet ID: 1249403815696257024 Status: created
Indexed Tweet ID: 1249403815742365699 Status: created
Indexed Tweet ID: 1249403816195141635 Status: created
Indexed Tweet ID: 1249403816228708354 Status: created
Indexed Tweet ID: 1249403816518094848 Status: created
Indexed Tweet ID: 1249403816488964097 Status: created
Indexed Tweet ID: 1249403816681840650 Status: created
Indexed Tweet ID: 1249403816618921984 Status: created
Indexed Tweet ID: 1249403816862265349 Status: created
Indexed Tweet ID: 1249403816933502976 Status: created
Indexed Tweet ID: 1249403816983826437 Status: created
Indexed Tweet ID: 1249403817185206272 Status: created
Indexed Tweet ID: 1249403817113853956 Status: created
Indexed Tweet ID: 1249403818166509568 Status: created
Indexed Tweet ID: 1249403818221199360 Status: created
Indexed Tweet ID: 1249403818229534720 Status: created
Indexed Tweet ID: 1249403818

Indexed Tweet ID: 1249403858738008064 Status: created
Indexed Tweet ID: 1249403860235325440 Status: created
Indexed Tweet ID: 1249403860260536320 Status: created
Indexed Tweet ID: 1249403860944326658 Status: created
Indexed Tweet ID: 1249403861766270976 Status: created
Indexed Tweet ID: 1249403861837758464 Status: created
Indexed Tweet ID: 1249403862630518784 Status: created
Indexed Tweet ID: 1249403862789828611 Status: created
Indexed Tweet ID: 1249403862567485440 Status: created
Indexed Tweet ID: 1249403862953234432 Status: created
Indexed Tweet ID: 1249403862823383042 Status: created
Indexed Tweet ID: 1249403863330959360 Status: created
Indexed Tweet ID: 1249403863670669312 Status: created
Indexed Tweet ID: 1249403864291233792 Status: created
Indexed Tweet ID: 1249403864853483520 Status: created
Indexed Tweet ID: 1249403864995909634 Status: created
Indexed Tweet ID: 1249403865096531968 Status: created
Indexed Tweet ID: 1249403864945582080 Status: created
Indexed Tweet ID: 1249403865

Indexed Tweet ID: 1249403906939068416 Status: created
Indexed Tweet ID: 1249403907438239744 Status: created
Indexed Tweet ID: 1249403908578897921 Status: created
Indexed Tweet ID: 1249403908755263489 Status: created
Indexed Tweet ID: 1249403896562319361 Status: created
Indexed Tweet ID: 1249403900479799299 Status: created
Indexed Tweet ID: 1249403908973375489 Status: created
Indexed Tweet ID: 1249403909459922947 Status: created
Indexed Tweet ID: 1249403909866586112 Status: created
Indexed Tweet ID: 1249403909346455552 Status: created
Indexed Tweet ID: 1249403910839660544 Status: created
Indexed Tweet ID: 1249403910898532352 Status: created
Indexed Tweet ID: 1249403910953095170 Status: created
Indexed Tweet ID: 1249403911292817409 Status: created
Indexed Tweet ID: 1249403912014065664 Status: created
Indexed Tweet ID: 1249403902421680128 Status: created
Indexed Tweet ID: 1249403913008267264 Status: created
Indexed Tweet ID: 1249403913167659009 Status: created
Indexed Tweet ID: 1249403913

Indexed Tweet ID: 1249403955588878337 Status: created
Indexed Tweet ID: 1249403956008288257 Status: created
Indexed Tweet ID: 1249403956045860864 Status: created
Indexed Tweet ID: 1249403956494729222 Status: created
Indexed Tweet ID: 1249403956746309632 Status: created
Indexed Tweet ID: 1249403956771635201 Status: created
Indexed Tweet ID: 1249403956880539648 Status: created
Indexed Tweet ID: 1249403956826009600 Status: created
Indexed Tweet ID: 1249403957203648512 Status: created
Indexed Tweet ID: 1249403957694341120 Status: created
Indexed Tweet ID: 1249403957845413889 Status: created
Indexed Tweet ID: 1249403958143127558 Status: created
Indexed Tweet ID: 1249403958134747140 Status: created
Indexed Tweet ID: 1249403958323339264 Status: created
Indexed Tweet ID: 1249403958398865408 Status: created
Indexed Tweet ID: 1249403958499495936 Status: created
Indexed Tweet ID: 1249403958738784259 Status: created
Indexed Tweet ID: 1249403959409815556 Status: created
Indexed Tweet ID: 1249403959

...

Indexed Tweet ID: 1249408736814301186 Status: created
Indexed Tweet ID: 1249408736893992960 Status: created
Indexed Tweet ID: 1249408736961146881 Status: created
Indexed Tweet ID: 1249408737040687104 Status: created
Indexed Tweet ID: 1249408737191899136 Status: created
Indexed Tweet ID: 1249408738143977473 Status: created
Indexed Tweet ID: 1249408738735214592 Status: created
Indexed Tweet ID: 1249408738852773894 Status: created
Indexed Tweet ID: 1249408740463235074 Status: created
Indexed Tweet ID: 1249408741923082241 Status: created
Indexed Tweet ID: 1249408742086639619 Status: created
Indexed Tweet ID: 1249408742006931457 Status: created
Indexed Tweet ID: 1249408742317330439 Status: created
Indexed Tweet ID: 1249408742598328326 Status: created
Indexed Tweet ID: 1249408742870978561 Status: created
Indexed Tweet ID: 1249408743025995777 Status: created
Indexed Tweet ID: 1249408743189741571 Status: created
Indexed Tweet ID: 1249408743080521728 Status: created
Indexed Tweet ID: 1249408743

Indexed Tweet ID: 1249408784264564747 Status: created
Indexed Tweet ID: 1249408784709169153 Status: created
Indexed Tweet ID: 1249408784881025027 Status: created
Indexed Tweet ID: 1249408784998346753 Status: created
Indexed Tweet ID: 1249408785678045184 Status: created
Indexed Tweet ID: 1249408786386821120 Status: created
Indexed Tweet ID: 1249408786520891399 Status: created
Indexed Tweet ID: 1249408786525118464 Status: created
Indexed Tweet ID: 1249408786613190656 Status: created
Indexed Tweet ID: 1249408787221565441 Status: created
Indexed Tweet ID: 1249408787959574530 Status: created
Indexed Tweet ID: 1249408787913400320 Status: created
Indexed Tweet ID: 1249408788265750528 Status: created
Indexed Tweet ID: 1249408788479619073 Status: created
Indexed Tweet ID: 1249408788827906049 Status: created
Indexed Tweet ID: 1249408788832178178 Status: created
Indexed Tweet ID: 1249408788798435330 Status: created
Indexed Tweet ID: 1249408788857270278 Status: created
Indexed Tweet ID: 1249408789

Indexed Tweet ID: 1249408827876954112 Status: created
Indexed Tweet ID: 1249408828120104961 Status: created
Indexed Tweet ID: 1249408828690456576 Status: created
Indexed Tweet ID: 1249408829105803267 Status: created
Indexed Tweet ID: 1249408829193957377 Status: created
Indexed Tweet ID: 1249408829340536833 Status: created
Indexed Tweet ID: 1249408829541974016 Status: created
Indexed Tweet ID: 1249408830280261634 Status: created
Indexed Tweet ID: 1249408830363975681 Status: created
Indexed Tweet ID: 1249408830330503169 Status: created
Indexed Tweet ID: 1249408830494015488 Status: created
Indexed Tweet ID: 1249408830913556480 Status: created
Indexed Tweet ID: 1249408831060353024 Status: created
Indexed Tweet ID: 1249408831282499589 Status: created
Indexed Tweet ID: 1249408831358205954 Status: created
Indexed Tweet ID: 1249408831626424320 Status: created
Indexed Tweet ID: 1249408831798546432 Status: created
Indexed Tweet ID: 1249408831920185347 Status: created
Indexed Tweet ID: 1249408832

Indexed Tweet ID: 1249408868653793281 Status: created
Indexed Tweet ID: 1249408868699996166 Status: created
Indexed Tweet ID: 1249408868884635652 Status: created
Indexed Tweet ID: 1249408868775530498 Status: created
Indexed Tweet ID: 1249408869111156737 Status: created
Indexed Tweet ID: 1249408869127917568 Status: created
Indexed Tweet ID: 1249408869954203648 Status: created
Indexed Tweet ID: 1249408870688026624 Status: created
Indexed Tweet ID: 1249408870688215043 Status: created
Indexed Tweet ID: 1249408871329869824 Status: created
Indexed Tweet ID: 1249408871371804678 Status: created
Indexed Tweet ID: 1249408871006900224 Status: created
Indexed Tweet ID: 1249408871879360512 Status: created
Indexed Tweet ID: 1249408872848056322 Status: created
Indexed Tweet ID: 1249408873070542849 Status: created
Indexed Tweet ID: 1249408873850699781 Status: created
Indexed Tweet ID: 1249408873859080196 Status: created
Indexed Tweet ID: 1249408874626637827 Status: created
Indexed Tweet ID: 1249408874

Indexed Tweet ID: 1249408914757730305 Status: created
Indexed Tweet ID: 1249408914808090624 Status: created
Indexed Tweet ID: 1249408915210723328 Status: created
Indexed Tweet ID: 1249408915105816578 Status: created
Indexed Tweet ID: 1249408915193974789 Status: created
Indexed Tweet ID: 1249408916129304576 Status: created
Indexed Tweet ID: 1249408916066381824 Status: created
Indexed Tweet ID: 1249408916150255617 Status: created
Indexed Tweet ID: 1249408916276023296 Status: created
Indexed Tweet ID: 1249408916443799553 Status: created
Indexed Tweet ID: 1249408916569624579 Status: created
Indexed Tweet ID: 1249408916674318341 Status: created
Indexed Tweet ID: 1249408916750041094 Status: created
Indexed Tweet ID: 1249408916766801925 Status: created
Indexed Tweet ID: 1249408917198774277 Status: created
Indexed Tweet ID: 1249408917484036096 Status: created
Indexed Tweet ID: 1249408918196871168 Status: created
Total runtime of the program is 750.3318951129913 seconds
All tweets have been ind

In [30]:
prepared_tweet

{'created_at': 'Sun Apr 12 18:27:26 +0000 2020',
 'id': 1249403770435493888,
 'id_str': '1249403770435493888',
 'text': 'RT @biannagolodryga: This image.\nThis quote.\n\n“One of the reasons the pandemic is hitting the country so hard is that no one appears to be…',
 'user': {'id': 4707764075,
  'id_str': '4707764075',
  'name': 'UpsidedownTurtle 🧢',
  'screen_name': 'Adakisn'},
 'retweeted_status': {'created_at': 'Sun Apr 12 12:40:06 +0000 2020',
  'id': 1249316363681910784,
  'id_str': '1249316363681910784',
  'text': 'This image.\nThis quote.\n\n“One of the reasons the pandemic is hitting the country so hard is that no one appears to… https://t.co/fypNKXxlpb',
  'user': {'id': 14135350,
   'id_str': '14135350',
   'name': 'Bianna Golodryga',
   'screen_name': 'biannagolodryga'},
  'retweet_count': 3085,
  'favorite_count': 6343,
  'favorited': False,
  'retweeted': False,
  'possibly_sensitive': False},
 'retweet_count': 0,
 'favorite_count': 0,
 'favorited': False,
 'retweeted': Fal

In [6]:
import json

# Open and read the file
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", 'r') as file:

    line_number = 0
    for line in file:
        line_number += 1
        # Skip empty lines or lines that do not contain valid JSON
        if not line.strip():
            continue
        try:
            tweet = json.loads(line)
            # Use tweet ID as the document ID in Elasticsearch
            res = es.index(index=index_name, id=tweet["id"], document=tweet)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON at line {line_number}: {e}")


In [7]:
def search_tweets(query_string):
    query = {
        "query": {
            "match": {
                "text": query_string
            }
        },
        "_source": ["created_at", "user.name", "text"]  # Specify the fields you want to retrieve
    }
    results = es.search(index=index_name, body=query)
    for hit in results['hits']['hits']:
        # Extracting and printing the specified fields from each tweet
        tweet_data = hit['_source']
        print(f"Username: {tweet_data['user']['name']}, Created At: {tweet_data['created_at']}, Text: {tweet_data['text']}")

# Example search
search_tweets("corona")


Username: Gordiota, Created At: Sun Apr 12 18:32:19 +0000 2020, Text: @simplicio10 CORONA CORONA CORONA CORONA
Username: Evliyaoğlu, Created At: Sun Apr 12 18:37:03 +0000 2020, Text: @sahraslan34 ➰Sen hiç duymadınmı Corona varrr corona corona😂🤗🤗
Username: Cah Lampung, Created At: Sun Apr 12 18:43:29 +0000 2020, Text: RT @JanaeAdriana1: corona oh corona wkwkw
Username: Ceren, Created At: Sun Apr 12 18:40:29 +0000 2020, Text: Oy corona corona çıktın hükümet avına
Username: Surya, Created At: Sun Apr 12 18:27:52 +0000 2020, Text: @PhotoChhoree Go corona go, go corona go.
Username: Neelendra thakur, Created At: Sun Apr 12 18:33:13 +0000 2020, Text: Puri Duniya yahan CORONA Se pareshan hai...!!

              Aur... Hum ..!!!

CORONA ki callertune Se !!

#Corona
Username: Ed Williams, Created At: Sun Apr 12 18:39:30 +0000 2020, Text: corona virus corona virus corona-So I misspelled corona and many other words.  I haven't misspelled this, "Halo cor… https://t.co/461k6jeyfY
Username: full tim

In [11]:
query = {
        "query": {
            "match": {
                "text": "corona"
            }
        },
        "_source": ["created_at", "user.name", "text"]  # Specify the fields you want to retrieve
    }
results = es.search(index=index_name, body=query)
results = es.search(index=index_name)

In [20]:
    for hit in results['hits']['hits']:
        # Extracting and printing the specified fields from each tweet
        tweet_data = hit['_source']
        break

In [21]:
tweet_data

{'created_at': 'Sun Apr 12 18:30:25 +0000 2020',
 'id': 1249404522662895618,
 'id_str': '1249404522662895618',
 'text': 'RT @ZeyniAKD1453: Pensilvanyada vaka gid gide cogaliyor Senin Radarinin Antenine Tukureyim Corona Ara Bul Şu Orospu cocugunu sana guveniyor…',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1229865631463485440,
  'id_str': '1229865631463485440',
  'name': 'İbrahim Aycıl',
  'screen_name': 'brahimAycl2',
  'location': None,
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1206,
  'friends_count': 1826,
  'listed_count': 0,
  'favourites_count': 12412,
  'statuses_count': 5830,
  'created_at': 'Tue Feb 18 20:30:09 +0000 2020',
  'utc_offs

In [2]:
# this code block allows you to inspect 2 tweets and 2 retweets in the dataset
# it also saves all the unique tweets/retweets in the dictionary tweets
# this code will help you to understand the data
# feel free to print out more fields

lines=0
num_tweets=0
num_retweets=0
print_tweet = 0
print_retweet = 0
tweets = {}

#insert path and replace name of the file below as needed
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            if ( data['text'].startswith('RT') ):
                num_retweets += 1
                #print out some fields of one retweet 
                #note that you should look at other fields too
                if (print_retweet < 2):
                    print_retweet += 1
                    print('RETWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    
                    print('User-- ',data['user'],'\n')
                  
                    print('retweetstatus--',data['retweeted_status'],'\n')
                  

            else:
                num_tweets += 1
                #print out some fields of one tweet 
                #note that you should look at other fields too
                if (print_tweet < 2):
                    print_tweet += 1
                    print('TWEET\n','id--',data['id'],'text--',data['text'],'\n')
                    print('User-- ',data['user'],'\n')
                    


            if (data['id_str'] not in tweets):
                tweets[data['id_str']] = data
            
        except:
            # if there is an error loading the json of the tweet, skip
            continue
            
print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
print('num of unique tweets/retweets=',len(tweets.keys()))

RETWEET
 id-- 1249403767180668930 text-- RT @nuffsaidny: wishing death on people is weirdo behavior. 

User--  {'id': 1242817830946508801, 'id_str': '1242817830946508801', 'name': 'juwelz v', 'screen_name': 'juwelz_v', 'location': 'Lower East Side, Manhattan', 'url': 'https://linktr.ee/juwelzv', 'description': 'Event Lyfe LLC .. Brand Ambassador: #visionarysociety Music Manager: @ssizzzlee & @tmoneybeatsbang Production🎥📸 #BringNyCultureBack 🌊🌊🌊 !!', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 43, 'friends_count': 118, 'listed_count': 0, 'favourites_count': 722, 'statuses_count': 906, 'created_at': 'Wed Mar 25 14:17:28 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': '', 'profile_background_image_url_https': '', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile

num of lines= 18518 num of tweets= 7356 num of retweets= 11162
num of unique tweets/retweets= 18506


In [14]:
# print the last data item
data

{'created_at': 'Sun Apr 12 18:47:53 +0000 2020',
 'id': 1249408918196871168,
 'id_str': '1249408918196871168',
 'text': 'RT @liputan6dotcom: Lagi, 2 Dokter Senior Meninggal karena Corona Covid-19 https://t.co/eDbMsxFA84',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 2283278160,
  'id_str': '2283278160',
  'name': 'Mr rius',
  'screen_name': 'AwhexWibowo',
  'location': 'Yogyakarta, Indonesia',
  'url': None,
  'description': 'have fun go mad',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 451,
  'friends_count': 608,
  'listed_count': 2,
  'favourites_count': 4625,
  'statuses_count': 31160,
  'created_at': 'Thu Jan 09 08:34:03 +0000 2014',
  'utc_offset': None,
  'time_zone': None,
  'g

In [15]:
data['user']

{'id': 2283278160,
 'id_str': '2283278160',
 'name': 'Mr rius',
 'screen_name': 'AwhexWibowo',
 'location': 'Yogyakarta, Indonesia',
 'url': None,
 'description': 'have fun go mad',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 451,
 'friends_count': 608,
 'listed_count': 2,
 'favourites_count': 4625,
 'statuses_count': 31160,
 'created_at': 'Thu Jan 09 08:34:03 +0000 2014',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'C0DEED',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_text_color': '333333',
 'profile_use_background_image': True,
 'pr

In [16]:
# this is a partial template for processing the dataset to store into the datastores.
# you will need to write functions for the psuedocode sections and invoke them here

#insert path and replace name of the file below as needed
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            # pseudocode: 
            # if tweet has been seen before, 
            #         continue (ignore this tweet, go to next iteration of loop)
            
            user = data['user']
            # psuedocode: 
            # if user has not been seen before
            #            add to user table 
            # update datastores with any metrics that you are tracking 
            
            if ( data['text'].startswith('RT') ):
                # psuedocode:
                # update retweet information
                # note that you may not have an entry for the original tweet 
                # if that is not in the dataset
                pass  # does nothing right now
                
            else:
                # psuedocode:
                # add the new tweet to datastore
                pass # does nothing right now
        except:
            continue


In [ ]:
# this is a partial template for processing the dataset to store into the datastores.
# you will need to write functions for the psuedocode sections and invoke them here

#insert path and replace name of the file below as needed
with open("/Users/layakishorereddy/Desktop/Rutgers/Dbms/Final Project/corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            # pseudocode: 
            # if tweet has been seen before, 
            #         continue (ignore this tweet, go to next iteration of loop)
            
            user = data['user']
            # psuedocode: 
            # if user has not been seen before
            #            add to user table 
            # update datastores with any metrics that you are tracking 
            
            if ( data['text'].startswith('RT') ):
                # psuedocode:
                # update retweet information
                # note that you may not have an entry for the original tweet 
                # if that is not in the dataset
                pass  # does nothing right now
                
            else:
                # psuedocode:
                # add the new tweet to datastore
                pass # does nothing right now
        except:
            continue
